In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
from utils import (create_dataset_LSTM, 
                    preprocess_data_LSTM,
                    create_and_train_model_LSTM)

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Favorita

In [2]:
# Leitura dos dados tratados
favorita = pd.read_parquet('Bases Tratadas/favorita_item_loja_semana.parquet')
favorita['Data'] = pd.to_datetime(favorita['Data'])

display(favorita)

,Produto,Loja,Data,Vendas
0,96995,2,2013-01-07,2
33584,830624,15,2013-01-07,13
33585,830624,16,2013-01-07,52
33586,830624,17,2013-01-07,11
33587,830624,18,2013-01-07,4
...,...,...,...,...
23380695,903283,45,2017-08-14,377
23380696,903283,46,2017-08-14,60
23380697,903283,47,2017-08-14,114
23380687,903283,34,2017-08-14,113


## Todas as lojas e produtos

In [3]:
# Agrupamento de vendas por data
favorita_agrupado = favorita[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
favorita_agrupado.set_index('Data', inplace=True)
display(favorita_agrupado)

,Vendas
Data,
2013-01-07,2546580
2013-01-14,2330352
2013-01-21,2384817
2013-01-28,2245352
2013-02-04,2534680
...,...
2017-07-17,5758675
2017-07-24,5738533
2017-07-31,5987120


In [4]:
# Pré-processamento dos dados
# Dividir os dados
train, test, scaler = preprocess_data_LSTM(
    favorita_agrupado['Vendas'].values, train_size_ratio=0.7
)

# Definir o look_back
look_back = 10

# Criar conjuntos de treinamento
X_train, y_train = create_dataset_LSTM(train, look_back)

# Estender o conjunto de teste para incluir os últimos `look_back` dados do conjunto de treinamento
extended_test = np.concatenate([train[-look_back:], test])

# Criar o conjunto de teste com base no conjunto estendido
X_test, y_test = create_dataset_LSTM(extended_test, look_back)

# Criar e treinar o modelo
start_train_time = time.time()
model, history = create_and_train_model_LSTM(X_train, y_train, look_back, epochs=1000)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = model.predict(X_test)
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Reverter a normalização das previsões e dos valores reais
test_predict_original = scaler.inverse_transform(test_predict).flatten()
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Garantir que os índices estão corretos
test_index = favorita_agrupado.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
train_index = favorita_agrupado.index[:len(train)]

print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Plotando os dados originais
fig = go.Figure()

# Add traces for real and predicted sales
fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')))
fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')))

# Add titles and labels
fig.update_layout(
    title='Vendas Reais vs Previstas',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

# Show the figure
fig.show()


3/3 [==============================] - 1s 8ms/step
Tempo de treino: 33.35716795921326 segundos
Tempo de previsão: 0.5523176193237305 segundos


In [5]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test_index,
    'Real': y_test_original,
    'Previsto': test_predict_original
})

# Salvar o dataframe em um arquivo CSV
df_resultado.to_csv('Resultados/Previsao/Favorita/Tudo/LSMT.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/Favorita/Tudo/LSMT.csv', index=False)

## Por loja e produtos relevantes

In [6]:
lojas_relevantes_favorita = [3, 8, 44, 45, 46, 47, 48, 49, 50, 51]
produtos_relevantes_favorita = {
    3: [1047679, 1503844],
    8: [1503844, 1047679],
    44: [1047679, 1503844],
    45: [1503844, 1473474],
    46: [584028, 1463992],
    47: [1503844, 1047679],
    48: [584028, 1463992],
    49: [1503844, 1473474],
    50: [584028, 364606],
    51: [1503844, 257847]
}

In [7]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_favorita])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_favorita):

    df_loja = favorita[favorita['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()

    df_loja_grouped.set_index('Data', inplace=True)
    
    train, test, scaler = preprocess_data_LSTM(
        df_loja_grouped['Vendas'].values, train_size_ratio=0.7
    )

    # Definir o look_back
    look_back = 10

    # Criar conjuntos de treinamento e validação
    X_train, y_train = create_dataset_LSTM(train, look_back)

    # Estender o conjunto de teste para incluir os últimos `look_back` dados do conjunto de validação
    extended_test = np.concatenate([train[-look_back:], test])

    # Criar o conjunto de teste com base no conjunto estendido
    X_test, y_test = create_dataset_LSTM(extended_test, look_back)

    # Ajustar o formato para [amostras, passos de tempo, características]
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    # Criar e treinar o modelo
    start_train_time = time.time()
    model, history = create_and_train_model_LSTM(X_train, y_train, look_back, epochs=1000)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time
    tempos_treinamento[loja] = train_time

    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = model.predict(X_test)
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time
    tempos_previsao[loja] = forecast_time

    # Reverter a normalização das previsões e dos valores reais
    test_predict_original = scaler.inverse_transform(test_predict).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Garantir que os índices estão corretos
    test_index = df_loja_grouped.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
    train_index = df_loja_grouped.index[:len(train)]

    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': y_test_original,
        'Previsto': test_predict_original
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)

        # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)
    # Criar um dataframe com o tempo de treino e previsão por loja



# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV

df_resultado_loja.to_csv('Resultados/Previsao/Favorita/Loja/LSMT.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/Favorita/Loja/LSMT.csv', index=False)

# Show the figure
fig.show()

# Print training and forecasting times
print("Tempos de treinamento por loja:", tempos_treinamento)
print("Tempos de previsão por loja:", tempos_previsao)


3/3 [==============================] - 0s 8ms/step


Tempos de treinamento por loja: {3: 13.41311240196228, 8: 7.843974828720093, 44: 13.697627782821655, 45: 9.751720190048218, 46: 10.770878553390503, 47: 6.030289888381958, 48: 14.718011140823364, 49: 5.769979476928711, 50: 12.033841848373413, 51: 13.463241338729858}
Tempos de previsão por loja: {3: 0.4895031452178955, 8: 0.48526811599731445, 44: 0.4853086471557617, 45: 0.5020720958709717, 46: 0.5017921924591064, 47: 0.5083277225494385, 48: 0.5019285678863525, 49: 0.5186343193054199, 50: 0.5187146663665771, 51: 0.5020802021026611}


In [8]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_favorita:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_favorita.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):
        df_loja_item = favorita[(favorita['Loja'] == loja) & (favorita['Produto'] == item)]
        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        df_loja_item = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
        df_loja_item.set_index('Data', inplace=True)
        
        train, test, scaler = preprocess_data_LSTM(
            df_loja_item['Vendas'].values, train_size_ratio=0.7
        )

        # Definir o look_back
        look_back = 10
        # Criar conjuntos de treinamento e teste
        X_train, y_train = create_dataset_LSTM(train, look_back)

        extended_test = np.concatenate([train[-look_back:], test])

        X_test, y_test = create_dataset_LSTM(extended_test, look_back)

        # Ajustar o formato para [amostras, passos de tempo, características]
        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

        # Criar e treinar o modelo
        start_train_time = time.time()
        model, history = create_and_train_model_LSTM(X_train, y_train, look_back, epochs=1000)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time
        tempos_treinamento[loja] = train_time

        # Realizar previsões no conjunto de teste
        start_forecast_time = time.time()
        test_predict = model.predict(X_test)
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time
        tempos_previsao[loja] = forecast_time

        # Reverter a normalização das previsões e dos valores reais
        test_predict_original = scaler.inverse_transform(test_predict).flatten()
        y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

        # Garantir que os índices estão corretos
        test_index = df_loja_item.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
        train_index = df_loja_item.index[:len(train)]

        df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
            'Data': test_index,
            'Loja': loja,
            'Item': item,
            'Real': y_test_original,
            'Previsto': test_predict_original
        })])

        df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })], ignore_index=True)

        # Add traces for train, test, and forecast
        fig_loja.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')), row=idx+1, col=1)
        
        # Update layout for each subplot
        fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)

    # Update layout for the entire figure
    fig_loja.update_layout(
        title=f'Sales Forecast for Store {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    # Show the figure
    fig_loja.show()

# Salvar o dataframe em um arquivo CSV
df_resultado_loja_item.to_csv('Resultados/Previsao/Favorita/Item/LSMT.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/Favorita/Item/LSMT.csv', index=False)

print(f"Tempos de treinamento:", tempos_treinamento)
print(f"Tempos de previsão:", tempos_previsao)


2/2 [==============================] - 0s 692us/step


3/3 [==============================] - 0s 0s/step


2/2 [==============================] - 0s 0s/step


2/2 [==============================] - 0s 17ms/step


2/2 [==============================] - 0s 0s/step


...

2/2 [==============================] - 0s 0s/step


2/2 [==============================] - 0s 772us/step


3/3 [==============================] - 1s 8ms/step


3/3 [==============================] - 0s 0s/step


Tempos de treinamento: {3: 11.172309875488281, 8: 5.937009572982788, 44: 11.188585758209229, 45: 10.904077291488647, 46: 10.12873649597168, 47: 7.994262933731079, 48: 12.343046426773071, 49: 11.629204511642456, 50: 17.846015214920044, 51: 7.395784854888916}
Tempos de previsão: {3: 0.5020015239715576, 8: 0.5021181106567383, 44: 0.5017731189727783, 45: 0.5019528865814209, 46: 0.5034599304199219, 47: 0.5186848640441895, 48: 0.5020389556884766, 49: 0.4963674545288086, 50: 0.5183444023132324, 51: 0.5149075984954834}


# M5

In [9]:
file_path_m5 = 'Bases Tratadas/m5_item_loja_semana.parquet'
df_m5 = pd.read_parquet(file_path_m5)
display(df_m5.head())

,Produto,Loja,Data,Vendas
0,HOBBIES_1_001,CA_1,1,0
1,HOBBIES_1_002,CA_1,1,0
2,HOBBIES_1_003,CA_1,1,0
3,HOBBIES_1_004,CA_1,1,0
4,HOBBIES_1_005,CA_1,1,0


## Todas as lojas e produtos

In [10]:
# Agrupamento de vendas por data
m5_agrupado = df_m5[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
m5_agrupado.set_index('Data', inplace=True)

display(m5_agrupado)

,Vendas
Data,
1,161932
2,194735
3,185293
4,170194
5,170777
...,...
273,287796
274,282731
275,314216


In [11]:
# Pré-processamento dos dados
# Dividir os dados
train, test, scaler = preprocess_data_LSTM(
    m5_agrupado['Vendas'].values, train_size_ratio=0.7
)

# Definir o look_back
look_back = 10

# Criar conjuntos de treinamento
X_train, y_train = create_dataset_LSTM(train, look_back)

# Estender o conjunto de teste para incluir os últimos `look_back` dados do conjunto de treinamento
extended_test = np.concatenate([train[-look_back:], test])

# Criar o conjunto de teste com base no conjunto estendido
X_test, y_test = create_dataset_LSTM(extended_test, look_back)

# Ajustar o formato para [amostras, passos de tempo, características]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Criar e treinar o modelo
start_train_time = time.time()
model, history = create_and_train_model_LSTM(X_train, y_train, look_back, epochs=1000)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = model.predict(X_test)
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Reverter a normalização das previsões e dos valores reais
test_predict_original = scaler.inverse_transform(test_predict).flatten()
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Garantir que os índices estão corretos
test_index = m5_agrupado.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
train_index = m5_agrupado.index[:len(train)]

print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Plotando os dados originais
fig = go.Figure()

# Add traces for real and predicted sales
fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')))
fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')))

# Add titles and labels
fig.update_layout(
    title='Vendas Reais vs Previstas',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

# Show the figure
fig.show()


3/3 [==============================] - 0s 0s/step
Tempo de treino: 11.051379442214966 segundos
Tempo de previsão: 0.5380494594573975 segundos


In [12]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test_index,
    'Real': y_test_original,
    'Previsto': test_predict_original
})

# Salvar o dataframe em um arquivo CSV
df_resultado.to_csv('Resultados/Previsao/M5/Tudo/LSMT.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/M5/Tudo/LSMT.csv', index=False)

## Por loja e produtos relevantes

In [13]:
lojas_relevantes_m5 = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

produtos_relevantes_m5 = {
    'CA_1': ['FOODS_3_090', 'FOODS_3_586'],
    'CA_2': ['FOODS_3_586', 'FOODS_3_252'],
    'CA_3': ['FOODS_3_090', 'FOODS_3_586'],
    'CA_4': ['FOODS_3_090', 'FOODS_3_586'],
    'TX_1': ['FOODS_3_586', 'FOODS_3_090'],
    'TX_2': ['FOODS_3_586', 'FOODS_3_090'],
    'TX_3': ['FOODS_3_586', 'FOODS_3_090'],
    'WI_1': ['FOODS_3_226', 'FOODS_3_694'],
    'WI_2': ['FOODS_3_226', 'FOODS_3_694'],
    'WI_3': ['FOODS_3_090', 'FOODS_3_226']
}

In [14]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_m5])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_m5):

    df_loja = df_m5[df_m5['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()

    df_loja_grouped.set_index('Data', inplace=True)
    
    train, test, scaler = preprocess_data_LSTM(
        df_loja_grouped['Vendas'].values, train_size_ratio=0.7
    )

    # Definir o look_back
    look_back = 10

    # Criar conjuntos de treinamento e teste
    X_train, y_train = create_dataset_LSTM(train, look_back)
    extended_test = np.concatenate([train[-look_back:], test])
    X_test, y_test = create_dataset_LSTM(extended_test, look_back)

    # Ajustar o formato para [amostras, passos de tempo, características]
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    # Criar e treinar o modelo
    start_train_time = time.time()
    model, history = create_and_train_model_LSTM(X_train, y_train, look_back, epochs=1000)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time
    tempos_treinamento[loja] = train_time

    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = model.predict(X_test)
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time
    tempos_previsao[loja] = forecast_time

    # Reverter a normalização das previsões e dos valores reais
    test_predict_original = scaler.inverse_transform(test_predict).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Garantir que os índices estão corretos
    test_index = df_loja_grouped.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste
    train_index = df_loja_grouped.index[:len(train)]

    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': y_test_original,
        'Previsto': test_predict_original
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)

    # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV
df_resultado_loja.to_csv('Resultados/Previsao/M5/Loja/LSMT.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/M5/Loja/LSMT.csv', index=False)
# Show the figure
fig.show()

# Print training and forecasting times
print("Tempos de treinamento por loja:", tempos_treinamento)
print("Tempos de previsão por loja:", tempos_previsao)


3/3 [==============================] - 0s 0s/step


Tempos de treinamento por loja: {'CA_1': 10.22720456123352, 'CA_2': 8.650939226150513, 'CA_3': 9.717053413391113, 'CA_4': 7.960959434509277, 'TX_1': 10.804397106170654, 'TX_2': 5.970040798187256, 'TX_3': 10.971234798431396, 'WI_1': 10.877108335494995, 'WI_2': 11.398164987564087, 'WI_3': 17.760785579681396}
Tempos de previsão por loja: {'CA_1': 0.5105328559875488, 'CA_2': 0.5246319770812988, 'CA_3': 0.5519952774047852, 'CA_4': 0.5353610515594482, 'TX_1': 0.5353074073791504, 'TX_2': 0.5338823795318604, 'TX_3': 0.5187344551086426, 'WI_1': 0.5204870700836182, 'WI_2': 0.8531908988952637, 'WI_3': 0.5034897327423096}


In [15]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_m5:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_m5.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):

        df_loja_item = df_m5[(df_m5['Loja'] == loja) & (df_m5['Produto'] == item)]

        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        df_loja_item = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
        df_loja_item.set_index('Data', inplace=True)
        
        train, test, scaler = preprocess_data_LSTM(
            df_loja_item['Vendas'].values, train_size_ratio=0.7
        )

        # Definir o look_back
        look_back = 10

        # Criar conjuntos de treinamento e teste
        X_train, y_train = create_dataset_LSTM(train, look_back)
        extended_test = np.concatenate([train[-look_back:], test])
        X_test, y_test = create_dataset_LSTM(extended_test, look_back)

        # Ajustar o formato para [amostras, passos de tempo, características]
        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

        # Criar e treinar o modelo
        start_train_time = time.time()
        model, history = create_and_train_model_LSTM(X_train, y_train,look_back, epochs=1000)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time
        tempos_treinamento[loja] = train_time

        # Realizar previsões no conjunto de teste
        start_forecast_time = time.time()
        test_predict = model.predict(X_test)
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time
        tempos_previsao[loja] = forecast_time

        # Reverter a normalização das previsões e dos valores reais
        test_predict_original = scaler.inverse_transform(test_predict).flatten()
        y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

        # Garantir que os índices estão corretos
        test_index = df_loja_item.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
        train_index = df_loja_item.index[:len(train)]

        df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
            'Data': test_index,
            'Loja': loja,
            'Item': item,
            'Real': y_test_original,
            'Previsto': test_predict_original
        })])

        df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })], ignore_index=True)

        # Add traces for train, test, and forecast
        fig_loja.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')), row=idx+1, col=1)
        
        # Update layout for each subplot
        fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)

    # Update layout for the entire figure
    fig_loja.update_layout(
        title=f'Sales Forecast for Store {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    # Show the figure
    fig_loja.show()

# Salvar o dataframe em um arquivo CSV
df_resultado_loja_item.to_csv('Resultados/Previsao/M5/Item/LSMT.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/M5/Item/LSMT.csv', index=False)

print(f"Tempos de treinamento:", tempos_treinamento)
print(f"Tempos de previsão:", tempos_previsao)


3/3 [==============================] - 0s 2ms/step


3/3 [==============================] - 1s 0s/step


3/3 [==============================] - 0s 365us/step


3/3 [==============================] - 0s 0s/step


3/3 [==============================] - 0s 0s/step


...

3/3 [==============================] - 1s 2ms/step


3/3 [==============================] - 1s 2ms/step


3/3 [==============================] - 1s 2ms/step


3/3 [==============================] - 1s 3ms/step


Tempos de treinamento: {'CA_1': 6.924161195755005, 'CA_2': 6.616478204727173, 'CA_3': 19.517974138259888, 'CA_4': 7.86052942276001, 'TX_1': 22.34454584121704, 'TX_2': 28.19815707206726, 'TX_3': 26.085196018218994, 'WI_1': 25.849385261535645, 'WI_2': 25.723543167114258, 'WI_3': 13.66062617301941}
Tempos de previsão: {'CA_1': 0.518648624420166, 'CA_2': 0.8029611110687256, 'CA_3': 0.5189299583435059, 'CA_4': 0.5019464492797852, 'TX_1': 0.5185036659240723, 'TX_2': 0.6290597915649414, 'TX_3': 0.5396368503570557, 'WI_1': 0.5715851783752441, 'WI_2': 0.5624654293060303, 'WI_3': 0.5670020580291748}


# Olist

In [16]:
file_path_olist = 'Bases Tratadas/olist_item_loja_semana.parquet'
df_olist = pd.read_parquet(file_path_olist)
display(df_olist.head())

,Produto,Loja,Data,Vendas
0,c1488892604e4ba5cff5b4eb4d595400,1554a68530182680ad5c8b042c3ab563,2016-09-05,1
1,f293394c72c9b5fafd7023301fc21fc2,1554a68530182680ad5c8b042c3ab563,2016-09-05,1
2,f3c2d01a84c947b078e32bbef0718962,a425f92c199eb576938df686728acd20,2016-09-05,1
3,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19,3
4,fd7fd78fd3cbc1b0a6370a7909c0a629,f09b760d23495ac9a7e00d29b769007c,2016-10-03,1


## Todas as lojas e produtos

In [17]:
# Agrupamento de vendas por data
olist_agrupado = df_olist[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
olist_agrupado.set_index('Data', inplace=True)
display(olist_agrupado)

,Vendas
Data,
2016-09-05,3
2016-09-19,3
2016-10-03,9
2016-10-10,354
2016-12-26,1
...,...
2018-07-30,1865
2018-08-06,2423
2018-08-13,2145


In [18]:
# Pré-processamento dos dados
# Dividir os dados
train, test, scaler = preprocess_data_LSTM(
    olist_agrupado['Vendas'].values, train_size_ratio=0.7
)

# Definir o look_back
look_back = 10

# Criar conjuntos de treinamento
X_train, y_train = create_dataset_LSTM(train, look_back)

# Estender o conjunto de teste para incluir os últimos `look_back` dados do conjunto de treinamento
extended_test = np.concatenate([train[-look_back:], test])

# Criar o conjunto de teste com base no conjunto estendido
X_test, y_test = create_dataset_LSTM(extended_test, look_back)

# Ajustar o formato para [amostras, passos de tempo, características]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Criar e treinar o modelo
start_train_time = time.time()
model, history = create_and_train_model_LSTM(X_train, y_train, look_back, epochs=1000)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = model.predict(X_test)
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Reverter a normalização das previsões e dos valores reais
test_predict_original = scaler.inverse_transform(test_predict).flatten()
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Garantir que os índices estão corretos
test_index = olist_agrupado.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
train_index = olist_agrupado.index[:len(train)]

print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Plotando os dados originais
fig = go.Figure()

# Add traces for real and predicted sales
fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')))
fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')))

# Add titles and labels
fig.update_layout(
    title='Vendas Reais vs Previstas',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

# Show the figure
fig.show()


1/1 [==============================] - 1s 522ms/step
Tempo de treino: 11.275091648101807 segundos
Tempo de previsão: 0.5656147003173828 segundos


In [19]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test_index,
    'Real': y_test_original,
    'Previsto': test_predict_original
})

# Salvar o dataframe em um arquivo CSV
df_resultado.to_csv('Resultados/Previsao/Olist/Tudo/LSMT.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/Olist/Tudo/LSMT.csv', index=False)

## Pro loja e produto

In [20]:
lojas_relevantes_olist = [
    '6560211a19b47992c3666cc44a7e94c0', '4a3ca9315b744ce9f8e9374361493884', 
    '1f50f920176fa81dab994f9023523100', 'cc419e0650a3c5ba77189a1882b7556a', 
    'da8622b14eb17ae2831f4ac5b9dab84a', '955fee9216a65b617aa5c0531780ce60', 
    '1025f0e2d44d7041d6cf58b6550e0bfa', '7c67e1448b00f6e969d365cea6b010ab', 
    'ea8482cd71df3c1969d7b9473ff13abc', '7a67c85e85bb2ce8582c35f2203ad736'
]

produtos_relevantes_olist = {
    '6560211a19b47992c3666cc44a7e94c0': ['a92930c327948861c015c919a0bcb4a8', 'd017a2151d543a9885604dc62a3d9dcc'],
    '4a3ca9315b744ce9f8e9374361493884': ['99a4788cb24856965c36a24e339b6058', '35afc973633aaeb6b877ff57b2793310'],
    '1f50f920176fa81dab994f9023523100': ['422879e10f46682990de24d770e7f83d', '389d119b48cf3043d311335e499d9c6b'],
    'cc419e0650a3c5ba77189a1882b7556a': ['154e7e31ebfa092203795c972e5804a6', '2b4609f8948be18874494203496bc318'],
    'da8622b14eb17ae2831f4ac5b9dab84a': ['e0cf79767c5b016251fe139915c59a26', 'fc1d8637c0268af3db482c14b7ef8e75'],
    '955fee9216a65b617aa5c0531780ce60': ['aca2eb7d00ea1a7b8ebd4e68314663af', '54d9ac713e253fa1fae9c8003b011c2a'],
    '1025f0e2d44d7041d6cf58b6550e0bfa': ['b532349fe46b38fbc7bb3914c1bdae07', '9ecadb84c81da840dbf3564378b586e9'],
    '7c67e1448b00f6e969d365cea6b010ab': ['8ed094bfe076c568f6bb10feada3f75d', '3eef0cb94ba82de806bb30ab743c7655'],
    'ea8482cd71df3c1969d7b9473ff13abc': ['97017430754804328eb9597b7f85da03', 'c1f5307decb89342351bec53668cffd9'],
    '7a67c85e85bb2ce8582c35f2203ad736': ['c6dd917a0be2a704582055949915ab32', '601a360bd2a916ecef0e88de72a6531a']
}

In [21]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_olist])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_olist):

    df_loja = df_olist[df_olist['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()
    date_range = pd.date_range(start=df_olist['Data'].min(), end=df_olist['Data'].max(), freq='W')
    date_range = date_range + pd.DateOffset(days=1)
    df_loja_grouped = df_loja_grouped.set_index('Data').reindex(date_range, fill_value=0).rename_axis('Data').reset_index()
    df_loja_grouped.set_index('Data', inplace=True)
    
    train, test, scaler = preprocess_data_LSTM(
        df_loja_grouped['Vendas'].values, train_size_ratio=0.7
    )

    # Definir o look_back
    look_back = 10

    # Criar conjuntos de treinamento e teste
    X_train, y_train = create_dataset_LSTM(train, look_back)
    extended_test = np.concatenate([train[-look_back:], test])
    X_test, y_test = create_dataset_LSTM(extended_test, look_back)

    # Ajustar o formato para [amostras, passos de tempo, características]
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    # Criar e treinar o modelo
    start_train_time = time.time()
    model, history = create_and_train_model_LSTM(X_train, y_train, look_back, epochs=1000)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time
    tempos_treinamento[loja] = train_time

    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = model.predict(X_test)
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time
    tempos_previsao[loja] = forecast_time

    # Reverter a normalização das previsões e dos valores reais
    test_predict_original = scaler.inverse_transform(test_predict).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Garantir que os índices estão corretos
    test_index = df_loja_grouped.index[-len(y_test):]
    train_index = df_loja_grouped.index[:len(train)]

    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': y_test_original,
        'Previsto': test_predict_original
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)

    # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV
df_resultado_loja.to_csv('Resultados/Previsao/Olist/Loja/LSMT.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/Olist/Loja/LSMT.csv', index=False)
# Show the figure
fig.show()

# Print training and forecasting times
print("Tempos de treinamento por loja:", tempos_treinamento)
print("Tempos de previsão por loja:", tempos_previsao)


1/1 [==============================] - 1s 526ms/step


Tempos de treinamento por loja: {'6560211a19b47992c3666cc44a7e94c0': 12.182565212249756, '4a3ca9315b744ce9f8e9374361493884': 11.440083026885986, '1f50f920176fa81dab994f9023523100': 11.570807218551636, 'cc419e0650a3c5ba77189a1882b7556a': 11.927858114242554, 'da8622b14eb17ae2831f4ac5b9dab84a': 11.729930877685547, '955fee9216a65b617aa5c0531780ce60': 12.054204940795898, '1025f0e2d44d7041d6cf58b6550e0bfa': 11.99887466430664, '7c67e1448b00f6e969d365cea6b010ab': 12.438509702682495, 'ea8482cd71df3c1969d7b9473ff13abc': 12.13164472579956, '7a67c85e85bb2ce8582c35f2203ad736': 12.200911521911621}
Tempos de previsão por loja: {'6560211a19b47992c3666cc44a7e94c0': 0.5753288269042969, '4a3ca9315b744ce9f8e9374361493884': 0.5875375270843506, '1f50f920176fa81dab994f9023523100': 0.5655519962310791, 'cc419e0650a3c5ba77189a1882b7556a': 0.561596155166626, 'da8622b14eb17ae2831f4ac5b9dab84a': 0.6360108852386475, '955fee9216a65b617aa5c0531780ce60': 0.5557050704956055, '1025f0e2d44d7041d6cf58b6550e0bfa': 0.622714

In [22]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_olist:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_olist.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):

        df_loja_item = df_olist[(df_olist['Loja'] == loja) & (df_olist['Produto'] == item)]

        if df_loja_item.empty:
            continue  # Ignorar se não houver dados

        df_loja_item = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
        date_range = pd.date_range(start=df_olist['Data'].min(), end=df_olist['Data'].max(), freq='W')
        date_range = date_range + pd.DateOffset(days=1)
        df_loja_item = df_loja_item.set_index('Data').reindex(date_range, fill_value=0).rename_axis('Data').reset_index()
        df_loja_item.set_index('Data', inplace=True)
        
        train, test, scaler = preprocess_data_LSTM(
            df_loja_item['Vendas'].values, train_size_ratio=0.7
        )

        # Definir o look_back
        look_back = 10
        try:
            # Criar conjuntos de treinamento e teste
            X_train, y_train = create_dataset_LSTM(train, look_back)
            extended_test = np.concatenate([train[-look_back:], test])
            X_test, y_test = create_dataset_LSTM(extended_test, look_back)

            # Ajustar o formato para [amostras, passos de tempo, características]
            X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
            X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

            # Criar e treinar o modelo
            start_train_time = time.time()
            model, history = create_and_train_model_LSTM(X_train, y_train, look_back, epochs=1000)
            end_train_time = time.time()
            train_time = end_train_time - start_train_time
            tempos_treinamento[loja] = train_time

            # Realizar previsões no conjunto de teste
            start_forecast_time = time.time()
            test_predict = model.predict(X_test)
            end_forecast_time = time.time()
            forecast_time = end_forecast_time - start_forecast_time
            tempos_previsao[loja] = forecast_time

            # Reverter a normalização das previsões e dos valores reais
            test_predict_original = scaler.inverse_transform(test_predict).flatten()
            y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

            # Garantir que os índices estão corretos
            test_index = df_loja_item.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
            train_index = df_loja_item.index[:len(train)]

            df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
                'Data': test_index,
                'Loja': loja,
                'Item': item,
                'Real': y_test_original,
                'Previsto': test_predict_original
            })])

            df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
                'Loja': [loja],
                'Item': [item],
                'Treino': [train_time],
                'Previsao': [forecast_time]
            })], ignore_index=True)

            # Add traces for train, test, and forecast
            fig_loja.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')), row=idx+1, col=1)
            fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=idx+1, col=1)
            fig_loja.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')), row=idx+1, col=1)
            
            # Update layout for each subplot
            fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
            fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)
        except:
            continue

    # Update layout for the entire figure
    fig_loja.update_layout(
        title=f'Sales Forecast for Store {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    # Show the figure
    fig_loja.show()

# Salvar o dataframe em um arquivo CSV
df_resultado_loja_item.to_csv('Resultados/Previsao/Olist/Item/LSMT.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/Olist/Item/LSMT.csv', index=False)

print(f"Tempos de treinamento:", tempos_treinamento)
print(f"Tempos de previsão:", tempos_previsao)


1/1 [==============================] - 1s 557ms/step


1/1 [==============================] - 1s 536ms/step


1/1 [==============================] - 1s 613ms/step


1/1 [==============================] - 1s 504ms/step


1/1 [==============================] - 1s 505ms/step


...

1/1 [==============================] - 1s 538ms/step


1/1 [==============================] - 1s 577ms/step


1/1 [==============================] - 1s 501ms/step


1/1 [==============================] - 1s 568ms/step


Tempos de treinamento: {'6560211a19b47992c3666cc44a7e94c0': 12.297360181808472, '4a3ca9315b744ce9f8e9374361493884': 12.85215449333191, '1f50f920176fa81dab994f9023523100': 12.256474256515503, 'cc419e0650a3c5ba77189a1882b7556a': 10.71645212173462, 'da8622b14eb17ae2831f4ac5b9dab84a': 10.00635838508606, '955fee9216a65b617aa5c0531780ce60': 6.569930791854858, '1025f0e2d44d7041d6cf58b6550e0bfa': 13.469916343688965, '7c67e1448b00f6e969d365cea6b010ab': 11.533323287963867, 'ea8482cd71df3c1969d7b9473ff13abc': 10.35032868385315, '7a67c85e85bb2ce8582c35f2203ad736': 12.395659446716309}
Tempos de previsão: {'6560211a19b47992c3666cc44a7e94c0': 0.606503963470459, '4a3ca9315b744ce9f8e9374361493884': 0.5677878856658936, '1f50f920176fa81dab994f9023523100': 0.645688533782959, 'cc419e0650a3c5ba77189a1882b7556a': 0.5360157489776611, 'da8622b14eb17ae2831f4ac5b9dab84a': 0.5402083396911621, '955fee9216a65b617aa5c0531780ce60': 0.5086245536804199, '1025f0e2d44d7041d6cf58b6550e0bfa': 0.5690064430236816, '7c67e1448